# 🧪 Laboratorio 5: Creación de un Pipeline de MLOps en Databricks

## 🎯 Objetivo

Crear un pipeline de MLOps que incluya entrenamiento, registro y despliegue automatizado de un modelo usando MLflow, Databricks Repos y Azure DevOps.

## ℹ️ Introducción

MLOps combina Machine Learning con prácticas de DevOps para asegurar versionado, trazabilidad y automatización en modelos de ML.
En este laboratorio usaremos MLflow para manejar el ciclo de vida del modelo y una integración básica con DevOps para automatizar el despliegue.

## 🧠 Conceptos Clave

**MLflow**
Plataforma open-source integrada en Databricks para gestión de experimentos, modelos y despliegue.
[📘 Ver documentación](https://mlflow.org/docs/latest/index.html)

**Databricks Repos**
Funcionalidad para conectar repositorios Git a Databricks y sincronizar notebooks o scripts.
[📘 Ver documentación](https://learn.microsoft.com/en-us/azure/databricks/repos/)

**Azure DevOps**
Plataforma de CI/CD y control de versiones. En este laboratorio simularemos una pipeline de despliegue desde el lado de Databricks.
[📘 Ver documentación](https://learn.microsoft.com/en-us/azure/devops/?view=azure-devops)

## 🛠️ Pasos del laboratorio

### 1. Preparar datos de entrenamiento (dataset Iris)

In [0]:
from sklearn.datasets import load_iris
import pandas as pd

# Cargar datos Iris
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

df.head()


### 2. Entrenar modelo con MLflow tracking

In [0]:
mlflow.end_run()

In [0]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1),
    df["target"],
    test_size=0.2,
    random_state=42
)

# Iniciar un experimento de MLflow
mlflow.set_experiment("/Users/alejandra.martinez@microsoft.com/iris_experiment")

with mlflow.start_run():
    clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, predictions)

    # Registrar parámetros y métricas
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 5)
    mlflow.log_metric("accuracy", acc)

    # Registrar el modelo
    mlflow.sklearn.log_model(clf, "modelo_rf")

    print(f"Accuracy: {acc}")


### 3. Registrar el modelo entrenado en MLflow Registry

**4.1 Verifica el experimento y selecciona el run**

En el menú lateral de Databricks, ve a Experiments.

Busca el experimento que creaste (iris_experiment).

Haz clic sobre él para ver la lista de ejecuciones (runs).

Selecciona el run que contiene tu modelo entrenado.

En la pestaña Artifacts, haz clic en modelo_rf y luego en Register Model.

**4.2 Registra el modelo**

Al hacer clic en Register Model, elige:

Model: Crea uno nuevo llamado IrisClassifier.

Haz clic en Register.

### 4. Desplegar y consumir el modelo

- En la UI de MLflow, selecciona el modelo registrado y habilita el endpoint de inferencia en tiempo real.

In [0]:
import mlflow.pyfunc

# Ruta al modelo en el registry
model_name = "IrisClassifier"
model_stage = "Staging"  # o "Production" según tu flujo

# Cargar el modelo desde el registry
modelo = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_stage}")

# Crear un nuevo DataFrame de prueba
df_test = df.drop("target", axis=1).iloc[:5]
predicciones = modelo.predict(df_test)
predicciones


**5.2 Validar el uso del modelo en producción**

Puedes usar esta inferencia en cualquier pipeline, dashboard o endpoint REST si activas Model Serving.

Para activar Model Serving:

Ve a Models > IrisClassifier.

Haz clic en Serve this model y sigue los pasos para habilitar un endpoint.

**6.1 Crear y conectar un repositorio**

En GitHub:

Crea un nuevo repositorio:

Nombre: mlops-pipeline-iris

Inicialízalo con un README.md y .gitignore (Python)

**En Databricks:**

Ve al menú lateral > New > More > Git Folder

Pega la URL del repositorio de GitHub
